# intro

In this notebook we will walk step-by-step through the process of designing a relational database. We have already discussed the power of relational databases to easily store huge amounts of data and quickly retrieve query results. The key to this power is the database "schema" or structure. Decisions taken during the design process can have huge downstream impacts on the effectiveness of your storage solution, and so the importance of setting off on the right track cannot be overstated! 

# The Basics

At a very high level, a relational database is simply a collection of tables. Each table is usually thought of as representing an individual "entity type", like a customer or store. These tables contain rows and columns, much like those of a spreadsheet, that store all the data for that "entity". The columns are the features of the data (i.e. names, ages, shoe sizes, favourite cheeses etc.) and the rows individual examples, each with a unique key. For example, a customer table may look like this:

| ID    | name    | favourite_cheese |
| :---- | ------- | ---------------: |
| 1     | John    | emmental         |
| 2     | Emma    | Cheddar          |
| 3     | Brandy  | Gorgonzola       |

The design of a table is referred to as its "schema". This is some basic information that defines the columns that make up each table, what goes in each column and the rules that apply to that column. Using the above as an example, a simple schema might say:

    id INTEGER PRIMARY KEY, 
    name TEXT NOT NULL, 
    favourite_cheese TEXT DEFAULT NULL 
    
although you may be able to parse the meaning of some of these terms, the specifics of this language aren't too important at the moment; just note that the schema defines the content of each column of the table and the rules that apply to them. The magic of relational databases is that, with this pre-defined schema spelling out exactly what to expect in each column of a table, the data can be very efficiently stored and queried.

The language used above to define the database schema is an example of Structured Query Language or SQL (pronounced sequel). SQL is the most popular language used to give instructsion to relational database engines, several of which are named after SQL i.e. SQLite, MySQL, PostgreSQL. As we walk through the process of creating a database, all of the database commands will be written in SQL.

# The Task

As a model example, we will tackle the very real problem of taking a large dataset and designing a database schema to effectively store it. We will be using The Movies Dataset - a group of files containing information from the GROUPLENS MOVIELENS PROJECT, which has been helpfully assembled in THIS KAGGLE DATASET. To save some time cleaning up the raw data and keeping focus on the task at hand, a few adjustments have been made to the raw dataset as it's found on Kaggle, prior to this notebook. Should anybody be interested in this preprocessing, it can be found in THIS PRE-PROCESSING NOTEBOOK.  

The (pre-processed) dataset consists of 3 CSV files:

* `movies.csv`: basic metadata for 44k+ movies - title, runtime, budget, original language etc.
* `ratings.csv`: 29 million + user reviews consisting of a simple 0.5 to 5 rating in increments of 0.5 
* `credits.csv`: The full casting credits of each movie, including the actor, charachter name, order of appearance etc.

We'll walk through the different files one-by-one and build our database as we go, introducing different principles and concepts as the are required. 

To build our database, we'll be using SQLite3 - a hugely popular and super simple database engine that has a fully-fledged python module provided right out of the box. We'll also be using the pandas data-analysis library to load and manipulate data. Whilst it is certainly worthwhile familiarising yourself with both, don't be to concerned if you don't have any experience with either - the real focus of this task is the SQL commands used to create the database. Each SQL database engine differs slightly in features and syntax, but the commands used in this notebook should be pretty universal. Every effort has been made to provide informative comments for all of the pandas script, so it should hopefully be easy to follow.

## Movies

First off, we'll take a look at the movies data:

In [1]:
import ast
from IPython.core.display import display, HTML
import numpy as np 
import os
import pandas as pd
import sqlite3

# load the movies csv file into a pandas DataFrame object
movies = pd.read_csv("movies_data/movies.csv")
# display some basic info from the movies DataFrame
movies.info()

In [3]:
# display the first 5 rows of the movies DataFrame
movies.head()

,id,title,original_title,runtime,budget,genres,original_language,overview,release_date,revenue,tagline
0,862,Toy Story,Toy Story,81.0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...",1995-10-30,373554033.0,NaN
1,8844,Jumanji,Jumanji,104.0,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,When siblings Judy and Peter discover an encha...,1995-12-15,262797249.0,Roll the dice and unleash the excitement!
2,15602,Grumpier Old Men,Grumpier Old Men,101.0,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,A family wedding reignites the ancient feud be...,1995-12-22,0.0,Still Yelling. Still Fighting. Still Ready for...
3,31357,Waiting to Exhale,Waiting to Exhale,127.0,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,"Cheated on, mistreated and stepped on, the wom...",1995-12-22,81452156.0,Friends are the people who let you be yourself...
4,11862,Father of the Bride Part II,Father of the Bride Part II,106.0,0,"[{'id': 35, 'name': 'Comedy'}]",en,Just when George Banks has recovered from his ...,1995-02-10,76578911.0,Just When His World Is Back To Normal... He's ...


We can see `movies.csv` contains the metadata for just over 44 thousand movies, as described above. 

When deciding how to structure our database, we should first consider what constitutes an appropriate "entity" to store in a table of our dataframe. `movies.csv` provides a great example of how to parse this concept, as the majority of the information is unique to, or relates specifically to, each movie. There is one exception here - although it many not be so obvious given the truncated table above. If we take a closer look at the `genres` column we see the following:

In [4]:
# adjust the pandas display settings so it displays full columns
with pd.option_context("display.max_colwidth", 1000):
    # show the first five rows again, but just the title and genre
    display(movies[["title", "genres"]].head())

,title,genres
0,Toy Story,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"
1,Jumanji,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]"
2,Grumpier Old Men,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"
3,Waiting to Exhale,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"
4,Father of the Bride Part II,"[{'id': 35, 'name': 'Comedy'}]"


It is clearer now that the genres column is a list of the genres that apply to each movie. These genres are categories that are not unique to each entry - we can see from the examples above that four out of the five are consiered comedies. As such, for the purposes of creating our database, we can consider the genre it's own "entity" and create a separate table to store this information. We'll put the genre's feature to one side and come back to it later:

In [5]:
# create a separate dataframe to store the genres feature for later
genres = (movies[["id", "genres"]]
          .rename(columns={"id":"movie_id"}))
# remove the genres feature from the movies dataframe
movies.drop("genres", axis=1, inplace=True)

With the genres set aside, we now wish to store all of our movies data in a database table. This leads us to consider some basic concepts of database design: 

### Data Types

To start, we need to consider the type of data we are storing in each column, as this has to be specified as part of the database schema. Each different SQL database engine offers a different range of datatypes, so your decisions here may differ depending on the tools available to you. SQLite3 offers the following 5 basic datatypes:

* **NULL** - just your run of the mill null
* **INTEGER** - a signed (+/-) integer in 1-8 bytes depending on its size
* **REAL** - an 8 byte floating point number
* **TEXT** - a text string
* **BLOB** - everything else! A "blob" of data stored exactly as it comes

The majority of our features should fit nicely with these datatypes. This may seem a little restrictive, however, when it comes to other data i.e. it's not clear how to store the `release_date` feature. Dont worry! SQLite offers compatibility with a huge range of datatype names that don't appear above, and "coerces" the data into the correct type as it's inserted. For example, dates can be input as **DATE** or **DATETIME** and SQLite will know what to do.

### Constraints

We should also consider some basic rules, or "constraints" as they are called in SQL. A good example is defining a **PRIMARY KEY** - every table needs one\*. A primary key is a unique identifier for each entry in your table, and helps facilitate queries and manipulations of your database during use. In the case of our movies data, this should clearly apply to the `ID` column.

\* *whilst historically this is accurate, it's not actually the case with SQLite3 or other modern SQL variants, but to keep things simple its easier to assume this requirement.*

There are literally hundreds of possible constraints that can be applied to columns or the whole table. Other common examples include:

* **NOT NULL** - specifies that Null values are not permitted in the column in question
* **UNIQUE** - enforces the rule that each item in the column is unique (no repeats)
* **DEFAULT** - assignes as default value to the column if one isn't provided

We could stay here for days listing the variants of schema design, but it's generally easiser to guide discussion with an example. With that in mind, lets get stuck in and define our movies table:

In [6]:
# create a connection object and a cursor to interact with our database
conn = sqlite3.connect("movies.db")
c = conn.cursor()
# change sqlite settings to include foreign keys functionality
c.execute("PRAGMA foreign_keys = ON")

# SQL command to create our movies table
create_movies_command = (
    """
    CREATE TABLE movies (
        id INTEGER PRIMARY KEY,
        title TEXT NOT NULL,
        original_title TEXT NOT NULL,
        runtime INTEGER NOT NULL,
        budget INTEGER NOT NULL,
        original_language TEXT NOT NULL,
        overview TEXT NOT NULL,
        release_date DATE NOT NULL,
        revenue INTEGER NOT NULL,
        tagline TEXT
    );
    """
)

# Have the database execute the create movies command
c.execute(create_movies_command);

The `create_movies_command` string above shows the typical format of a SQL **CREATE TABLE** command - hence the name! Let's pick it apart bit-by-bit:

* `CREATE TABLE movies`: This tells SQLite to create a table called movies (shocking!)
* `()`: the rest of the command including all the column definitions and constraints is contained within braces
* `id INT PRIMARY KEY`: this defines a column called `id` that will contain `INTEGERS` and tells SQLite that this is our `PRIMARY KEY` - the database engine will ensure that each entry in this column is unique
* `title TEXT NOT NULL`: this defines a column called `title` containing `TEXT` or strings that is `NOT NULL` i.e. the database engine will not permit this column to be left empty
* And so on.... It should be relatively simple to infer the meaning of the rest of the statements

**Note**: each SQL command is typed in ALL CAPS. This is a stylistic feature that is intended to help distinguish between SQL commands and user input variables (column/table names and the like); it isn't enforced by the SQLite or other database engines, but you will almost always see SQL statements written in this manner. Commands also finish with a `;`

And with that, we've built our movies table and we're ready to start filling it with data. The schema we've defined ensures that we aren't able to input data that breaks any of the rules we've set, such as leaving out any of the `NOT NULL` values or entering two movies with the same `id`. Now we'll have a go at entering, or `INSERT`ing, our first movie:

In [7]:
# get the first entry in the movies DataFrame and display it
first_movie = movies.iloc[0]
first_movie

id                                                                 862
title                                                        Toy Story
original_title                                               Toy Story
runtime                                                             81
budget                                                        30000000
original_language                                                   en
overview             Led by Woody, Andy's toys live happily in his ...
release_date                                                1995-10-30
revenue                                                    3.73554e+08
tagline                                                            NaN
Name: 0, dtype: object

In [8]:
insert_command = (
    """
    INSERT INTO movies (
        id, title, original_title, runtime,
        budget, original_language, overview,
        release_date, revenue, tagline
    )
    VALUES (
        862, "Toy Story", "Toy Story", 81, 30000000,
        "en", "Led by Woody....", "1995-10-30", 373554033,
        NULL
    );
    """
)
c.execute(insert_command);

Despite it being only our second SQL command, the `insert_command` should hopefully already feel a little familiar, and not require a super-detailed explanation. As before the SQL commands appear in ALL CAPS - clearly we are telling the database engine to `INSERT` data `INTO` the `movies` table. The columns we are adding information to are declared `(` inside braces `)`, and the values to be inserted follow the `VALUES` command `(` in their own braces `)`.

As a quick sanity-check, we can query the database for the entry we just inserted:

In [9]:
# ask  SQLite to return every entry and column in the movies table
# this should only be the one entry we just added!
c.execute("select * from movies").fetchall()

[(862,
  'Toy Story',
  'Toy Story',
  81,
  30000000,
  'en',
  'Led by Woody....',
  '1995-10-30',
  373554033,
  None)]

Et voila - our database is up and running! It would obviously take some time to manually input the thousands of movies in the database one-by-one using raw SQL commands, so we'll use the SQLite3 module and pandas to insert the rest of the entries programatically - we wont dwell on the details as they are beyond the scope of this notebook. How you go about bulk inserting data into your own database will really depend on the tools you have at your disposal.

In [10]:
# remove example movie added above from DataBase
# will cause an id conflit when adding all data again
c.execute("DELETE FROM movies WHERE id = 862")
# new SQL command using the sqlite3 module's parameter substitution feature
insert_movie_command = (
    """
    INSERT INTO movies (
        id, title, original_title, runtime,
        budget, original_language, overview,
        release_date, revenue, tagline
    )  
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?) 
    """
)
# iteratively execute insert command with tuples from movies DataFrame 
c.executemany(insert_movie_command, movies.itertuples(index=False))
# commit changes to dataframe
conn.commit();

And with our movies table full, we'll move onto the next set of data...
## Ratings

In [11]:
ratings = pd.read_csv("movies_data/ratings.csv")
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25976754 entries, 0 to 25976753
Data columns (total 4 columns):
user_id      int64
rating       float64
timestamp    int64
movie_id     float64
dtypes: float64(2), int64(2)
memory usage: 792.7 MB


In [12]:
ratings.head()

,user_id,rating,timestamp,movie_id
0,1,1.0,1425941529,197.0
1,1,4.5,1425942435,10474.0
2,1,5.0,1425941523,238.0
3,1,5.0,1425941546,240.0
4,1,5.0,1425941556,207.0


We can see that the ratings data contains a whopping 26 million entries! Fortunately, each entry seems pretty simple - just four numeric features. As with our movies table, we now need to decide how to store this data. The ratings data introduces us two a couple of new concepts:

### Composite Primary Keys

The first issue we face is defining a PRIMARY KEY. There isn't a single unique feature of the ratings data, like that of the `id` column in the movies table. A bit of data exploration will show that each user can submit only one rating per movie, so the **combination** of `user_id` and `movie_id` is unique. SQLite allows us to leverage this and define a composite primary key - that is, a PRIMARY KEY composed of two or more columns of our table (generally speaking, more than two is not sensible). When composing our INSERT command we define a composite PRIMARY KEY as a table constraint rather than a column constraint - we'll see an example of this in a moment.

### Foreign Keys

Foreign keys are what put the "relation" in relational database - they allow us to formalise the relationship between one or more columns found in different tables

. We do this by specifying a FOREIGN KEY: a column constraint telling our database that one of the columns in our table relates to a column in another table. In the case of the ratings data, each rating "belongs" to a specific movie. Therefore, the `movie_id` column of the ratings data corresponds with the `id` column of the movies table. This is what's called a "one to many" relationship: one movie can be related to many ratings, but each rating belongs to only one movie.

Foreign keys allow us to define rules about changing or deleting data in our database. This prevents us from "orphaning" entries in one table by deleting or changing related "parent" entries in another table; think of the reviews for "Toy Story" if we were to delete the coresponding movie or change its id - they would no longer "belong" to an existing movie, and so would be rating nothing! Insted we can tell our database what to do when a movie is changed or deleted. For example, we can tell the database to delete / update all corresponding ratings as their "parent" movie is deleted / updated, or simply prevent any changes to movies that have corresponding ratings.  

We'll go ahead and create a new table to store the ratings, using these new concepts:

In [13]:
create_ratings_command = (
    """
    CREATE TABLE ratings (
        user_id INT,
        rating INT NOT NULL,
        timestamp DATETIME NOT NULL,
        movie_id INT,
        PRIMARY KEY(user_id, movie_id),
        FOREIGN KEY (movie_id) REFERENCES movies (id)
            ON DELETE RESTRICT
            ON UPDATE CASCADE
    );
    """ 
)
c.execute(create_ratings_command);

The `create_ratings_command` begins in much the same way that we did when creating our movies table. Note that now, instead of defining one column as a `PRIMARY KEY`, we define it separately as a table constraint containing the `user_id` and `movie_id` columns inside brackets. We also define a `FOREIGN KEY` table constraint on the `movie_id` column, specifying that this `REFERENCES` the `id` column in the `movies` table. Using this key, we tell our database that `ON` attempting to `DELETE` a "parent" movie entry, the database should `RESTRICT` this request and return an error. Similarly, we state that `ON UPDATE`ing a "parent" movie entry the changes should `CASCADE` through any relevant ratings - put simply, if the movie id is changed this should be updated on any corresponding ratings.   

A couple of interesting points to note at this stage:

1. we don't need to specify that a PRIMARY KEY should be NOT NULL - SQLite enforces this for us

2. we've stored two very different forms of DATE in our database. The `release_date` in the movies table is a python strftime string of the form YYYY-MM-DD, in contrast to the `timestamp` in the ratings table, which is a "unix epoch timestamp" counting the number of seconds since 01/01/1970 (e.g. 1425941546). SQLite has datetime functions capable of parsing both these formats, meaning we can stick them both in our database without worrying about any issues when we come to query them.  

As before, we need to insert all of the ratings data into our new table. With 26 million entries, this one takes a little longer than the last, so a bit of patience is requred!

In [14]:
insert_ratings_command = (
    """
        INSERT INTO ratings (
            user_id, 
            rating, 
            timestamp, 
            movie_id
        )
        VALUES (?, ?, ?, ?)
    """ 
)

c.executemany(insert_ratings_command,
              ratings.itertuples(index=False))
conn.commit();

And with that, we have our second table and we've already dealt with the majority of our data! Now that we're familiar with FOREIGN KEYs, lets go back to the `genres` data that we put aside earlier:

## Genres

If you remember, we sidelined the genres that appeared in the movies data earlier, as they seemed to be an entity in their own right.

In [15]:
# adjust the pandas display settings so it displays full columns
with pd.option_context("display.max_colwidth", 1000):
    # show the first five rows again, but just the title and genre
    display(genres.sample(5))

,movie_id,genres
18342,142118,[]
30671,307479,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'name': 'Action'}]"
33797,13986,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]"
19300,55723,"[{'id': 27, 'name': 'Horror'}]"
41126,423122,"[{'id': 35, 'name': 'Comedy'}]"


Like the ratings data, each genre has to correspond with a move. Unlike the ratings data, each genre can relate to more than one movie, and each movie can have more than one genre. This is what we call a "many to many" relationship. Such a relationship presents us with an issue: if we store the genre's like we did the ratings, we'll end up with thousands of repeating entries saying "comedy" with different individual `movie_id`s and still more thousands saying "drama", and so on. To visualise this, we'll parse all of the data from the genres dataframe so that each entry has it's own row: 

In [16]:
# generate individual entries for each movie-genre pair
movie_genres = [] # empty list to contain movie-genres 
for entry in genres.itertuples():
    # generates list of dicts from json-like string 
    entry_genres = ast.literal_eval(entry.genres)
    # loops through list of dicts
    for genre in entry_genres:
        # adds movie_id to dict
        genre["movie_id"] = entry.movie_id
        # appends entry to movie_genres list
        movie_genres.append(genre)
# creates DataFrame from list of movie-genre dicts
movie_genres = pd.DataFrame(movie_genres)

# display the first 10 movie_genres
movie_genres.head(10)

,id,name,movie_id
0,16,Animation,862
1,35,Comedy,862
2,10751,Family,862
3,12,Adventure,8844
4,14,Fantasy,8844
5,10751,Family,8844
6,10749,Romance,15602
7,35,Comedy,15602
8,35,Comedy,31357
9,18,Drama,31357


As you can see, each genre is repeated as it corresponds with a movie. In this case, it may not be such an issue given were only repeating a short string, but imagine if each genre came with a lot of data, like images or lengthy descriptions. Repeating all this data this over and over wouldn't be practical. Also, what if we decided to change the description of a genre - we might need to ammend thousands of entries. Instead we need another solution:

### Junction Tables

Like the name suggests, a junction table defines a "juction" between one table and another. In the case of our genre's example, we can create a junction table that contains the `movie_id`s and `genre_id`s for each movie-genre combination. That way, we only record two id integers for each relation, and we have separate tables with only one entry for each genre and each movie. The `movie_genres` table above is already effectively a junction table - all we need is to put the `name` column in a separate genres table and we're set:

In [17]:
# create new genre's table with only one entry for each genre
genres = (movie_genres[["id", "name"]]
          .drop_duplicates())

# re_index genre_id as its a bit silly (20 categories with ids that go up to 10751)
# create new genre id from 0-19
genres['new_genre_id'] = np.arange(len(genres))
# merge this new id with the movie_genres junction table
movie_genres = movie_genres.merge(genres[['id','new_genre_id']], 
                                  on="id", 
                                  how="left")
# drop old id and name features from junction table
movie_genres.drop(["id", "name"], axis=1, inplace=True)
# rename genre_id feature
movie_genres.rename(columns={'new_genre_id':'genre_id'}, inplace=True)
# drop old id from genres table
genres.drop('id', axis=1, inplace=True)
# rename new id feature
genres.rename(columns={'new_genre_id':'id'}, inplace=True)

# display the title and first five rows of each dataframe
display(HTML("<strong>Genres:</strong>"))
display(genres.head(5))
display(HTML("<strong>Movie Genres:</strong>"))
display(movie_genres.head(5))

,name,id
0,Animation,0
1,Comedy,1
2,Family,2
3,Adventure,3
4,Fantasy,4


,movie_id,genre_id
0,862,0
1,862,1
2,862,2
3,8844,3
4,8844,4


Now we have our genres table, and our movie_genres junction table, lets create tables in our database to store this data:

In [18]:
create_genres_command = (
    """
    CREATE TABLE genres (
        id INT PRIMARY KEY,
        name TEXT NOT NULL
    );
    """
)
c.execute(create_genres_command)

create_movie_genres_command = (
    """
    CREATE TABLE movie_genres (
        movie_id INT,
        genre_id INT,
        PRIMARY KEY (movie_id, genre_id),
        FOREIGN KEY (movie_id) REFERENCES movies (id)
            ON DELETE RESTRICT
            ON UPDATE CASCADE,
        FOREIGN KEY (genre_id) REFERENCES genres (id)
            ON DELETE RESTRICT
            ON UPDATE CASCADE
    );
    """
)
c.execute(create_movie_genres_command);

The above SQL commands should hopefully now be familar. We first `CREATE` a simple `TABLE` called `genres` with an `id` column as the `PRIMARY KEY` and a `name` column for each genre. We then `CREATE` a junction `TABLE` called `movie_genres` - again this has two simple columns that form a composite `PRIMARY KEY (movie_id,` and `genre_id),`. As with all junction tables, the two columns are both a   `FOREIGN KEY` that `REFERENCES` the primary key of two other tables in the database, forming a jucntion between them. The rest should hopefully be familiar based on the principles we have already covered. 

Now lets get the relevant data into these two new tables and move on to our last csv file:

In [19]:
# insert all the genres data into the genres table
c.executemany("""
              INSERT INTO genres (name, id)
              VALUES (?, ?)
              """, 
              genres.itertuples(index=False))
# the same with the genres data
c.executemany("""
              INSERT INTO movie_genres (movie_id, genre_id)
              VALUES (?, ?)
              """, 
              movie_genres.itertuples(index=False))
conn.commit();

# credits

As the name suggests, the credits data stores the actor credits for each movie:

In [20]:
credits = pd.read_csv("movies_data/credits.csv")
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 2 columns):
movie_id    45476 non-null int64
credits     45476 non-null object
dtypes: int64(1), object(1)
memory usage: 710.7+ KB


In [21]:
# adjust the pandas display settings so it displays full columns
with pd.option_context("display.max_colwidth", 200):
    # show the first five rows again, but just the title and genre
    display(credits.head(5))

,movie_id,credits
0,862,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'},..."
1,8844,"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': 2, 'id': 2157, 'name': 'Robin Williams', 'order': 0, 'profile_path': '/sojtJyIV3lkUeThD7A2oHNm8183.j..."
2,15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a9251416c75077a8d', 'gender': 2, 'id': 6837, 'name': 'Walter Matthau', 'order': 0, 'profile_path': '/xJVkvprOnzP5Zdh5y63y8HHniDZ.jp..."
3,31357,"[{'cast_id': 1, 'character': ""Savannah 'Vannah' Jackson"", 'credit_id': '52fe44779251416c91011aad', 'gender': 1, 'id': 8851, 'name': 'Whitney Houston', 'order': 0, 'profile_path': '/69ouDnXnmklYPr4..."
4,11862,"[{'cast_id': 1, 'character': 'George Banks', 'credit_id': '52fe44959251416c75039eb9', 'gender': 2, 'id': 67773, 'name': 'Steve Martin', 'order': 0, 'profile_path': '/rI2EMvkfKKPKa5z0nM2pFVBtUyO.jp..."


We can see that the data is of a similar form to the genres data we just dealt with i.e. a list / array of individual credits. It's a little difficult to see all the details from the table above, as all the entries have been truncated. Lets quickly process the data into a table of credits that's a little easier to read:

In [22]:
all_credits = []
# loops through every entry in the credits dataframe
for entry in credits.itertuples():
    # captures the list of credits for the current movie
    entry_credits = ast.literal_eval(entry.credits)
    # loops through each credit for the current movie
    for credit in entry_credits:
        # adds the movie id to each individual credit
        credit["movie_id"] = entry.movie_id
        # add the credit to the all_credits list
        all_credits.append(credit)
# turns the all_credits list into DataFrame
all_credits = pd.DataFrame(all_credits)

# drop movie_ids that don't appear in movies
movie_ids = movies.id.values
relevant_credits = all_credits.movie_id.isin(movie_ids)
all_credits = all_credits[relevant_credits]

all_credits.sample(5)

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
328489,15,Romeo Tan,52fe4c2ec3a368484e1ab63b,2,18472,Kevin Corrigan,10,/trRjEABsF2BCqn6pib3bYhYIxNH.jpg,137321
367182,10,Hilda Fenchurch,52fe45fb9251416c910451ed,1,89528,Faye Emerson,0,/fsubYCmD4Xd60mZg20UNx90JRc3.jpg,36635
236393,2,Martin Beck,52fe44d8c3a368484e03a1cb,0,360116,Carl-Gustaf Lindstedt,0,/fVbkDu2seYWLJH6x1mPpQVm4V0m.jpg,25699
4746,67,Roger,57d3dbe4c3a368557a002b67,2,3206,Kirk Baltz,18,/zCuugN0Lgsi5IKRq9OyYl3vh9pU.jpg,241
403580,8,Drago,52fe442c9251416c91007527,2,44161,Richard Paul,3,/lHbaXaGWSnL0PPAKbJtjjHr2wkp.jpg,30643


Each credit contains information about an individual character from the movie it relates to, and the actor that played that character. Using the principles we have already learned: 

* it is clear that the character information has a one-to-many with the movies data - each movie has several characters but each character can be thought of as relating to only one movie (we'll ignore the case of series / franchises for the sake of simplicity)
* the actor information has a many-to-many relationship with each movie - each actor can appear in many movies, and many actors can appear in each movie

Based on this, we should create an actors table to store all the information relating to each actor. We can then create a characters table that stores all the character information, and that also acts as a junction table between the movies table and an actors table. This is a great demonstration that junction tables need not only provide a link between two datasets - they can often sore useful information in their own right.

Let's split our credits data into actors and credits:

In [23]:
# select actor-related info from credits and remove duplicates
actors = all_credits[["id", "name", "gender"]].drop_duplicates("id")
# drop all actor-related info from credits info
credits = (all_credits[["movie_id", "character", "id", "order"]]
           .rename(columns={"id": "actor_id"}))
# drop all duplicated credits - there are a few for some reason
credits.drop_duplicates(subset=["movie_id", "actor_id"], inplace=True)

# display the title and first five rows of each dataframe
display(HTML("<strong>Actors:</strong>"), actors.head(5))
display(HTML("<strong>Credits:</strong>"), credits.head(5))

,id,name,gender
0,31,Tom Hanks,2
1,12898,Tim Allen,2
2,7167,Don Rickles,2
3,12899,Jim Varney,2
4,12900,Wallace Shawn,2


,movie_id,character,actor_id,order
0,862,Woody (voice),31,0
1,862,Buzz Lightyear (voice),12898,1
2,862,Mr. Potato Head (voice),7167,2
3,862,Slinky Dog (voice),12899,3
4,862,Rex (voice),12900,4


and now that we've isolated the data we want to store, lets create tables for the data and insert it:

In [24]:
create_actors_command = (
    """
    CREATE TABLE actors (
        id INT PRIMARY KEY,
        name TEXT NOT NULL,
        gender INT
    );
    """
)
c.execute(create_actors_command)

insert_actors_command = (
    """
    INSERT INTO actors (id, name, gender)
    VALUES (?, ?, ?)
    """
)
c.executemany(insert_actors_command, 
              actors.itertuples(index=False));

In [26]:
create_credits_command = (
    """
    CREATE TABLE credits (
        movie_id INT, 
        character TEXT NOT NULL, 
        actor_id INT, 
        appearance_order INT NOT NULL, 
        PRIMARY KEY(movie_id, actor_id), 
        FOREIGN KEY (movie_id) 
        REFERENCES movies (id) 
            ON DELETE RESTRICT 
            ON UPDATE CASCADE, 
        FOREIGN KEY (actor_id) 
        REFERENCES actors (id) 
            ON DELETE RESTRICT 
            ON UPDATE CASCADE
    );
    """
)
c.execute(create_credits_command)

insert_credits_command = (
    """
    INSERT INTO credits (movie_id, character, actor_id, appearance_order)
    VALUES (?, ?, ?, ?)
    """
)
c.executemany(insert_credits_command, 
              credits.itertuples(index=False))
conn.commit()
conn.close();

Important Note: The `appearance_order` column was originally labeled `order` in the dataframe above. The name was chaged when creating the credits table, as SQLite has an `ORDER` function and reserves all function names to prevent any namespace clashes. Be mindful of this when chosing your column / table names as SQLite will return an error if there's a clash.

Thats us done - whew! We've taken all the data we started with and have built a SQLite database to store it. In the process, we've carefully considered the features of our data and how to organise them to best suit the features of a relational database. To recap, here are a few key considerations when creating a database:

1. **Splitting Up Data:** What does your data contain? Decide how your data divides into individual entities i.e. information that relates to the same concept and does not repeat unnecesarily. Create tables according to these entities. 
2. **Primary Keys:** What is the unique identifying feature of each entry? Select or define a unique feature or combination of features that can be used to identify each entry in your table
3. **Constraints:** What rules should be followed when adding entries? Choose appropriate column / table constraints to ensure the data in your tables is of the correct format and has the right features.
4. **Table relationships:** How do your entities / tables relate to one another? Carefully define one-to-many and many-to-many relations, create any required junction tables, and apply any foreign key constraints to ensure your database is managed properly 

And that about does it.

Quick postscript:

The above notebook has been structured around the historic basic principles of relational databases. Many modern SQL database engines, such as Oracle or Google BigQuery, offer users the option to "nest" data or repeat entries in a list-like format, like the genre and credits data we saw above. Such features would have a `STRUCT` or `RECORD` datatype. If you have these advanced datatypes at your disposal, your design decisions may be very different when building your database. Indeed, these features in their original format could have been stored in their original format using these more advanced datatypes, and were probably taken directly from a database with a schema built using these features!